# Tensor Methods Homework 4

### 1 CPD approximation by ALS

Implement the alternating-least-squares (ALS) algorithm for the low-rank approximation of a tensor of dimension $d \in \N$ at least two and mode sizes $n_1, \dots, n_d \in \N$. For $R, r \in \N$ such that $r < R$, the implementation should take, as input parameters, a rank-$R$ CPD of the tensor given in the form of matrices $U_k \in \R^{n_k \times R}$ with $k ∈ \{1, \dots, d\}$ and a rank-$r$ CPD of the initial guess given in the form of matrices $V_k \in \R^{n_k \times r}$ with $k \in \{1, \dots, d\}$. The number of iterations to be performed should be an input parameter.

The best-accuracy rank-$r$ least-squares approximation problem is based on the cost function given by
$$ \phi(V_1, \dots, V_d) = \| \psi_r(V_1, \dots, V_d) − \psi_R(U_1, \dots, U_d)\|_F $$
for all $V_k \in \R^{n_k \times r}$ with $k \in \{1, \dots, d\}$, where $\psi_r$ and $\psi_R$ denote the CPD multilinear representation maps transforming CP decompositions of ranks $r$ and $R$ into tensors $\R^{n_1 \times \dots \times n_d}$.

In your implementation, a single ALS iteration starting at a CPD given by $V_k \in \R^{n_k \times r}$ with $k \in \{1, \dots, d\}$ should consists in updating $V_k$ for fixed $V_1, \dots, V_{k−1}, V_{k+1}, \dots, V_d$ with a solution of the optimization problem
$$ \phi(V_1, \dots, V_d) \to \min_{V_k \in \R^{n_k \times r}} $$
sequentially for $k = 1, 2, . . . , d − 1, d, d − 1, d − 2, . . . , 3, 2$.

An additional requirement is as follows: the cost of a single iteration should be linear with respect to $d$ (certain elementwise products of Gram matrices accumulating from the left and from the right should be stored, and exactly one of these should be updated at each step of a single iteration).

Make sure that your implementation evaluates the above cost function $\phi$, the 2-norm of $\frac{1}{2} \nabla \phi^2$ and the two-norms of the $r$ CPD terms after each iteration and returns the history of the computed values alongside the final CP appproximation.

In [1]:
using LinearAlgebra

In [2]:
struct CPD{T<:Number}
    frames::Vector{Matrix{T}}
    shape::NTuple{N,Integer} where {N}
    rank::Integer
    function CPD{T}(frames::Vector{Matrix{T}}) where T<:Number
        shape = [size(U,1) for U in frames]
        ranks = [size(U,2) for U in frames]

        # non empty
        if isempty(frames)
            error("No frame matrices were provided.")
        end

        # rank dimensions have to match and be > 0
        if ranks[1] < 0
            error("Rank has to be greater than 0.")
        end
        if any(ranks .!= ranks[1])
            error("Frame matrices do not share the same rank dimension.")
        end
        
        # mode dimensions have to be > 0
        if any(shape .< 0)
            error("Mode dimensions have to be greater than 0.")
        end

        # create instance
        new{T}(frames, Tuple(shape), ranks[1])
    end
end

# defer element type from input
CPD(frames::Vector{Matrix{T}}) where {T<:Number} = CPD{T}(frames)

# basic functions
Base.eltype(::Type{<:CPD{T}}) where {T} = T
Base.ndims(A::CPD{T}) where {T} = length(A.frames)
Base.size(A::CPD{T}) where {T} = A.shape
Base.length(A::CPD{T}) where {T} = prod(size(A))
rank(A::CPD{T}) where{T} = A.rank
function Base.size(A::CPD{T}, k::Integer) where {T}
    if k < 0
        error("arraysize: dimension out of range")
    end
    if k <= ndims(A)
        return A.shape[k]
    else
        return 1
    end
end

# construct tensor from CPD, implemented by Prof. Kazeev
function totensor(A::CPD{T}) where {T}
    d = ndims(A)
    n = size(A)
    r = rank(A)
    S = zeros(T, prod(n))
    for α = 1:r
        B = A.frames[1][:,α]
        for k = 2:1:d
            B = kron(A.frames[k][:,α], B)
        end
        S .+= B
    end
    reshape(S, n...)
end

;

In [3]:
function Khatri_Rao(A::AbstractMatrix, B::AbstractMatrix)
    i, k = size(A)
    j, kb = size(B)
    @assert k == kb "Column dimensions do not match!"

    TA = eltype(A)
    TB = eltype(B)
    T = promote_type(TA, TB)
    C = similar(A, T, (i*j, k))

    for t = 1:k
        C[:, t] = kron(A[:,t], B[:,t])
    end
    return C
end

Khatri_Rao (generic function with 1 method)

In [23]:
function ALS(A::CPD, X0::CPD, max_iter::Integer)
    d = ndims(A)
    n = size(A)
    R = rank(A)
    r = rank(X0)
    TA = eltype(A)
    TX = eltype(X0)
    T = promote_type(TA, TX)

    @assert (d == ndims(X0)) && all(n == size(X0)) "Mode dimensions of initial guess do not match given tensor."
    @assert r < R "Rank of initial guess has to be smaller than that given tensor."
    @assert max_iter > 0 "Number of maximum iterations has to be greater than 0, got $max_iter"
    
    S = totensor(A)
    X = copy(X0.frames)
    norms = []
    
    for _ = 1:max_iter
        H_l = Matrix{T}(I, r, r)
        #H_r = foldl((.*), [X[i]' * X[i] for i = 2:d])
        for k = 1:d
            #H_l = k == 1 ? Matrix{T}(I, r, r) : foldl((.*), [X[i]' * X[i] for i = 1:(k-1)])
            H_l = H_l .* (X[k]' * X[k])
            H_r = k == d ? Matrix{T}(I, r, r) : foldl((.*), [X[i]' * X[i] for i = (k+1):d])
            #H_r = H_r ./ (X[k]' * X[k])
            H = H_l .* H_r

            if k == 1
                KR = foldl(Khatri_Rao, X[(k+1):d])
            elseif k == d
                KR = foldl(Khatri_Rao, X[1:(k-1)])
            else
                KR_left = foldl(Khatri_Rao, X[1:(k-1)])
                KR_right = foldl(Khatri_Rao, X[(k+1):d])
                KR = Khatri_Rao(KR_left, KR_right)
            end

            # unfolding matrix
            p = prod(n[1:k-1])
            q = prod(n[k+1:end])
            B = reshape(S, p, n[k], q)
            B = permutedims(B, [2,1,3])
            B = reshape(B, n[k], p*q)

            X[k] = B * KR * H'
            X[k] = X[k] / norm(X[k])
        end
        
        val = norm(totensor(CPD(X)) + S)
        push!(norms, val)
    end

    return CPD(X), norms
end

ALS (generic function with 1 method)

In [24]:
d = 3
n = (40,50,60)
R = 30
r = 20
U = Vector{Matrix{Float64}}(undef, d)
V = Vector{Matrix{Float64}}(undef, d)
for k = 1:d
    U[k] = rand(n[k], R)
    V[k] = rand(n[k], r)
end

A = totensor(CPD(U))
B_CPD, norms = ALS(CPD(U), CPD(V), 10)
B = totensor(B_CPD)
;

In [25]:
norms

10-element Vector{Any}:
 1331.4118994301425
 1332.1065808629373
 1332.1181281531951
 1332.118128153195
 1332.118128153195
 1332.118128153195
 1332.118128153195
 1332.118128153195
 1332.118128153195
 1332.118128153195

### 2 ALS approximation of the matrix-multiplication tensor

In [29]:
""" from master solution of homework 2 """

function matmultensor(::Type{T}, n::Int) where T<:Number
    @assert n ≥ 2
    S = zeros(T, n, n, n, n, n, n) # rowA, colA, rowB, colB, rowC, colC
    for i ∈ 1:n, j ∈ 1:n, k ∈ 1:n
        # C_{ij} = \sum_{k=1}^n A_{ik} B_{kj}
        S[i,k,k,j,i,j] = 1
    end
    reshape(S, n^2, n^2, n^2)
end

function matmultensorcpd(::Type{T}, n::Int) where T<:Number
    @assert n ≥ 2
    U = zeros(T, n, n, n, n, n)
    V = zeros(T, n, n, n, n, n)
    W = zeros(T, n, n, n, n, n)
    for i ∈ 1:n, j ∈ 1:n, k ∈ 1:n
        # C_{ij} = \sum_{k=1}^n A_{ik} B_{kj}
        U[i,k,i,j,k] = 1
        V[k,j,i,j,k] = 1
        W[i,j,i,j,k] = 1
    end
    U = reshape(U, n^2, n^3)
    V = reshape(V, n^2, n^3)
    W = reshape(W, n^2, n^3)
    [U,V,W]
end

matmultensorcpd (generic function with 1 method)

In [31]:
M2 = matmultensor(Float64, 2)
r2 = 7

M3 = matmultensor(Float64, 3)
r3 = 23

M4 = matmultensor(Float64, 4)
r4 = 49


4×4×4 Array{Float64, 3}:
[:, :, 1] =
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0

### 3 ALS approximation of a Laplace-like tensor tensor

In [26]:
a = [1, 0]
b = [0, 1]
U1 = hcat(b, a, a)
U2 = hcat(a, b, a)
U3 = hcat(a, a, b)
T = reshape(kron(b, a, a) + kron(a, b, a) + kron(a, a, b), 2, 2, 2) # Laplace-like tensor

# initial guess
V = Vector{Matrix{Float64}}(undef, 3)
for k = 1:3
    V[k] = rand(2, 2)
end

T_CPD, norms = ALS(CPD([U1, U2, U3]), CPD(V), 1000000);

In [27]:
norms

1000000-element Vector{Any}:
 2.289743494090558
 2.5105255286283126
 2.5116206529684812
 2.511813956347031
 2.511846884189029
 2.5118515443072584
 2.511852068470737
 2.5118521158561244
 2.5118521200953867
 2.511852120673332
 ⋮
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218
 2.511852120798218